read_csv中有个参数chunksize，

通过指定一个chunksize分块大小来读取文件，

返回的是一个可迭代的对象TextFileReader

In [ ]:
import pandas as pd
reader = pd.read_csv('tmp.csv', chunksize=4)

In [ ]:
for chunk in reader:
    print(chunk)

指定iterator=True 也可以返回一个可迭代对象TextFileReader ：

In [ ]:
reader = pd.read_table('tmp.sv', sep='|', iterator=True)

In [ ]:
reader.get_chunk(5)

我需要打开的数据集是个csv文件，大小为3.7G，

并且对于数据一无所知，所以首先打开前5行观察数据的类型，列标签等等：

In [ ]:
chunks = pd.read_csv('train.csv',iterator = True)
chunk = chunks.get_chunk(5)
print (chunk)

Preliminary
　　我们谈到“文本处理”时，我们通常是指处理的内容。Python 将文本文件的内容读入可以操作的字符串变量非常容易。文件对象提供了三个“读”方法： .read()、.readline() 和 .readlines()。每种方法可以接受一个变量以限制每次读取的数据量，但它们通常不使用变量。 .read() 每次读取整个文件，它通常用于将文件内容放到一个字符串变量中。然而.read() 生成文件内容最直接的字符串表示，但对于连续的面向行的处理，它却是不必要的，并且如果文件大于可用内存，则不可能实现这种处理。下面是read()方法示例：

In [ ]:
try:
    f = open('/path/to/file', 'r')
    print f.read()
finally:
    if f:
        f.close()

调用read()会一次性读取文件的全部内容，如果文件有10G，内存就爆了，所以，要保险起见，可以反复调用read(size)方法，每次最多读取size个字节的内容。另外，调用readline()可以每次读取一行内容，调用readlines()一次读取所有内容并按行返回list。因此，要根据需要决定怎么调用。
　　如果文件很小，read()一次性读取最方便；如果不能确定文件大小，反复调用read(size)比较保险；如果是配置文件，调用readlines()最方便：

In [ ]:
for line in f.readlines():
    process(line) # <do something with line>

Read In Chunks
　　处理大文件是很容易想到的就是将大文件分割成若干小文件处理，处理完每个小文件后释放该部分内存。这里用了iter & yield
  
  指定每次读取的长度

有时，可能希望对每次读取的内容进行更细粒度的控制。

在这种情况下，可以使用 iter 和 yield

In [ ]:
def read_in_chunks(filePath, chunk_size=1024*1024):
    """
    Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1M
    You can set your own chunk size
    """
    file_object = open(filePath)
    while True:
        chunk_data = file_object.read(chunk_size)
        if not chunk_data:
            break
        yield chunk_data

filePath = './path/filename'
for chunk in read_in_chunks(filePath):
    process(chunk) # <do something with chunk>

Using with open()

　　with语句打开和关闭文件，包括抛出一个内部块异常。for line in f文件对象f视为一个迭代器，会自动的采用缓冲IO和内存管理，所以你不必担心大文件。

In [ ]:
#If the file is line based
with open(...) as f:
    for line in f:
    process(line) # <do something with line>

简述

在处理大数据时，有可能会碰到好几个 G 大小的文件。如果通过一些工具（例如：NotePad++）打开它，会发生错误，无法读取任何内容。

那么，在 Python 中，如何快速地读取这些大文件呢？

一般的读取

In [ ]:
with open('filename', 'r', encoding = 'utf-8') as f:
    for line in f.readlines():
        do_something(line)

但是，当完成这一操作时，readlines() 方法（read() 也一样）会将整个文件加载到内存中。在文件较大时，往往会引发 MemoryError（内存溢出）。

那么，如何避免这个问题？

使用 fileinput 模块

稍微好点儿的方式是使用 fileinput 模块：

In [ ]:
import fileinput

for line in fileinput.input(['filename']):
    do_something(line)

调用 fileinput.input() 会按照顺序读取行，但是在读取之后不会将它们保留在内存中。

逐行读取

除此之外，也可使用 while() 循环和 readline() 来逐行读取：

In [ ]:
with open('filename', 'r', encoding = 'utf-8') as f:
    while True:
        line = f.readline()  # 逐行读取
        if not line:  # 到 EOF，返回空字符串，则终止循环
            break
        do_something(line)

自动管理

这才是 Pythonci 最完美的方式，既高效又快速

with 语句句柄负责打开和关闭文件（包括在内部块中引发异常时），for line in f 将文件对象 f 视为一个可迭代的数据类型，会自动使用 IO 缓存和内存管理，这样就不必担心大文件了。

In [ ]:
with open('filename', 'r', encoding = 'utf-8') as f:
    for line in f:
        do_something(line)

Pandas提供了IO工具可以将大文件分块读取，测试了一下性能，完整加载9800万条数据也只需要263秒左右，还是相当不错了

In [ ]:
importpandas as pd

reader =pd.read_csv('data/servicelogs', iterator=True)

try:

    df =reader.get_chunk(100000000)

exceptStopIteration:

    print"Iteration is stopped."

使用不同分块大小来读取再调用 pandas.concat 连接DataFrame，chunkSize设置在1000万条左右速度优化比较明显。

In [ ]:
loop =True

chunkSize =100000

chunks =[]

whileloop:

    try:

        chunk =reader.get_chunk(chunkSize)

        chunks.append(chunk)

    exceptStopIteration:

        loop =False

        print"Iteration is stopped."

df =pd.concat(chunks, ignore_index=True)

read_csv()函数的iterator参数等于True时，表示返回一个TextParser以便逐块读取文件；

chunkSize表示文件块的大小，用于迭代；

TextParser类的get_chunk方法用于读取任意大小的文件块；

StopIteration的异常表示在循环对象穷尽所有元素时报错；

concat()函数用于将数据做轴向连接：
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, Verify_integrity=False)
常用参数： 
objs：Series,DataFrame或者是Panel构成的序列list; 
axis：需要合并连接的轴，0是行，1是列； 
join：连接的参数，inner或outer； 
ignore=True表示重建索引。

In [ ]:
1. read() 接口的问题

f = open(filename, 'rb')
f.read()

我们来读取 1 个 nginx 的日至文件，规模为 3Gb 大小。read() 方法执行的操作，是一次性全部读入内存，显然会造成：

MemoryError
...

也即会发生内存溢出。

2. 解决方案：转换接口

（1）readlines() ：读取全部的行，构成一个 list，实践表明还是会造成内存的问题；

for line in f.reanlines():
    ...

（2）readline()：每次读取一行，

while True:
    line = f.readline()
    if not line:
        break

（3）read(1024)：重载，指定每次读取的长度

while True:
    block = f.read(1024)
    if not block:

3. 真正 Pythonic 的方法

真正 Pythonci 的方法，使用 with 结构：

with open(filename, 'rb') as f:
    for line in f:
        <do something with the line>

对可迭代对象 f，进行迭代遍历：for line in f，会自动地使用缓冲IO（buffered IO）以及内存管理，而不必担心任何大文件的问题。

In [5]:
import numpy as np
import random

In [6]:
samp  = [ i for i in sorted(random.sample(range(10),5))]
samp

[0, 1, 2, 6, 9]

In [7]:
np.random.sample(10)

array([0.08155842, 0.80155206, 0.48521373, 0.04952713, 0.99588369,
       0.00384505, 0.92578094, 0.577394  , 0.61168731, 0.81880164])